## The Joint - Part 2
#### By: Chris Green, Ph.D.

#### In this notebook, we used Foursquare to determine the number of Likes and Average user ratings for each local Joint clinic. Unfortunately, there was not enough data on Foursquare to definitively draw a conclusion relative to customer impression.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### Import Table generated from prior K-means Analysis

In [2]:
df_joint = pd.read_csv('Joint_Locations_National.csv', header = 0)
df_joint.drop(['Unnamed: 0'], axis=1, inplace=True)
df_joint


,Location,Street Address,Zip Code,Longitude,Latitude
0,The Joint Chiropractic - Gateway Crossing,"9925 West McDowell Road, Suite #102","Avondale, AZ 85392",-112.273244,33.464710
1,The Joint Chiropractic - Casa Grande,"1609 E. Florence Blvd., Suite 3","Casa Grande, AZ 85122",-111.717751,32.879560
2,The Joint Chiropractic - Ocotillo,"2895 S. Alma School Road, Suite #6","Chandler, AZ 85286",-111.858498,33.263999
3,The Joint Chiropractic - Chandler - Ahwatukee,"7131 W. Ray Rd., Suite 39","Chandler, AZ 85226",-111.964882,33.319773
4,The Joint Chiropractic - Flagstaff,"2700 S Woodlands Village Blvd, Ste. 440","Flagstaff, AZ 86001",-111.667484,35.175820
5,The Joint Chiropractic - San Tan Village,"2765 S. Market St., Ste 105","Gilbert, AZ 85295",-111.743563,33.300936
6,The Joint Chiropractic - Gilbert Town Square,1084 S. Gilbert Road #103,"Gilbert, AZ 85296",-111.790251,33.330003
7,The Joint Chiropractic - Arrowhead Ranch,"20280 N. 59th Avenue, Suite #107","Glendale, AZ 85308",-112.186889,33.669916
8,The Joint Chiropractic - Goodyear,"1468 N Litchfield Road, Suite M-6","Goodyear, AZ 85395",-112.358289,33.462838
9,The Joint Chiropractic - Dana Park,"3426 E. Baseline Road, Suite #116","Mesa, AZ 85204",-111.756936,33.379282


In [3]:
CLIENT_ID = 'SJDL5MPZ5XJPAYO0ZWJGU2IWTJXTG21V0Z40UVRVICX31I1K' # your Foursquare ID
CLIENT_SECRET = 'EOIXHXP3ZN12RP3DE3JBVZCNW30ZJ4GOZGIN2TAOMRWUBZ5S' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
radius =100
print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)



Your credentials:
CLIENT_ID: SJDL5MPZ5XJPAYO0ZWJGU2IWTJXTG21V0Z40UVRVICX31I1K
CLIENT_SECRET:EOIXHXP3ZN12RP3DE3JBVZCNW30ZJ4GOZGIN2TAOMRWUBZ5S


#### Create a Sample query to confirm appropriate form

In [4]:
# type your answer here
name = df_joint["Location"][0]
latitude = df_joint["Latitude"][0]
longitude = df_joint["Longitude"][0]
query = df_joint["Street Address"][0]
radius = 100
LIMIT = 2
url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION,latitude, longitude, name, radius, LIMIT)
url


'https://api.foursquare.com/v2/venues/search?&client_id=SJDL5MPZ5XJPAYO0ZWJGU2IWTJXTG21V0Z40UVRVICX31I1K&client_secret=EOIXHXP3ZN12RP3DE3JBVZCNW30ZJ4GOZGIN2TAOMRWUBZ5S&v=20180605&ll=33.46470954948393,-112.27324355398464&query=The Joint Chiropractic - Gateway Crossing&radius=100&limit=2'

#### Determine the Venue IDs for each Clinic

In [7]:
#joint_id =[]
#jt_loc = []
for ct in range(257,len(df_joint['Latitude'])):
    
    latitude = df_joint["Latitude"][ct]
    longitude = df_joint["Longitude"][ct]
    query = df_joint["Street Address"][ct]
    name = df_joint["Location"][ct]
    radius = 300
    LIMIT = 1

    url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION,latitude, longitude, name, radius, LIMIT) 

    results = requests.get(url).json()['response']['venues']
    if len(results) == 0:
        results = '000000' # Exception handling if no Venue ID is returned
    else:
        results = requests.get(url).json()['response']['venues'][0]['id']# Venue ID from foursquare
   
    joint_id.append(results)
    jt_loc.append(name)
                

In [8]:
ct

474

#### Creates a  Comprehensive DF with all extracted data including Foursquare Venue ID

In [10]:
df_joint['Venue_ID']=joint_id
df_joint

,Location,Street Address,Zip Code,Longitude,Latitude,Venue_ID
0,The Joint Chiropractic - Gateway Crossing,"9925 West McDowell Road, Suite #102","Avondale, AZ 85392",-112.273244,33.464710,5798eb94cd105f35ed7b2456
1,The Joint Chiropractic - Casa Grande,"1609 E. Florence Blvd., Suite 3","Casa Grande, AZ 85122",-111.717751,32.879560,5b3a5bac66f3cd0039a044f9
2,The Joint Chiropractic - Ocotillo,"2895 S. Alma School Road, Suite #6","Chandler, AZ 85286",-111.858498,33.263999,4f8709a3e4b09582a022e03c
3,The Joint Chiropractic - Chandler - Ahwatukee,"7131 W. Ray Rd., Suite 39","Chandler, AZ 85226",-111.964882,33.319773,587324e9d60c8b3454f0c92d
4,The Joint Chiropractic - Flagstaff,"2700 S Woodlands Village Blvd, Ste. 440","Flagstaff, AZ 86001",-111.667484,35.175820,000000
5,The Joint Chiropractic - San Tan Village,"2765 S. Market St., Ste 105","Gilbert, AZ 85295",-111.743563,33.300936,57366b8b498e4086cc2f8b70
6,The Joint Chiropractic - Gilbert Town Square,1084 S. Gilbert Road #103,"Gilbert, AZ 85296",-111.790251,33.330003,000000
7,The Joint Chiropractic - Arrowhead Ranch,"20280 N. 59th Avenue, Suite #107","Glendale, AZ 85308",-112.186889,33.669916,4bbfd52e74a9a593033dcff6
8,The Joint Chiropractic - Goodyear,"1468 N Litchfield Road, Suite M-6","Goodyear, AZ 85395",-112.358289,33.462838,58b8d5682f91cb60484ceac9
9,The Joint Chiropractic - Dana Park,"3426 E. Baseline Road, Suite #116","Mesa, AZ 85204",-111.756936,33.379282,4b4d17ccf964a520acca26e3


#### Code below determined the number of likes accumulated at each clinic

In [16]:
like_ct = []

for b in range(len(df_joint['Venue_ID'])):
    
    VENUE_ID = df_joint['Venue_ID'].iloc[b]
   
    if VENUE_ID != '000000':
        url = 'https://api.foursquare.com/v2/venues/{}/likes?&client_id={}&client_secret={}&v={}'.format(VENUE_ID,CLIENT_ID, CLIENT_SECRET, VERSION)
        results = requests.get(url).json()['response']['likes']['count']
        like_ct.append(results)
    else:
        results=0
        like_ct.append(results)
        
    print(b)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

#### Add the total likes for each Venue to our working DataFrame

In [18]:
df_joint['Like Count']= like_ct
df_joint

,Location,Street Address,Zip Code,Longitude,Latitude,Venue_ID,Like Count
0,The Joint Chiropractic - Gateway Crossing,"9925 West McDowell Road, Suite #102","Avondale, AZ 85392",-112.273244,33.464710,5798eb94cd105f35ed7b2456,0
1,The Joint Chiropractic - Casa Grande,"1609 E. Florence Blvd., Suite 3","Casa Grande, AZ 85122",-111.717751,32.879560,5b3a5bac66f3cd0039a044f9,0
2,The Joint Chiropractic - Ocotillo,"2895 S. Alma School Road, Suite #6","Chandler, AZ 85286",-111.858498,33.263999,4f8709a3e4b09582a022e03c,0
3,The Joint Chiropractic - Chandler - Ahwatukee,"7131 W. Ray Rd., Suite 39","Chandler, AZ 85226",-111.964882,33.319773,587324e9d60c8b3454f0c92d,0
4,The Joint Chiropractic - Flagstaff,"2700 S Woodlands Village Blvd, Ste. 440","Flagstaff, AZ 86001",-111.667484,35.175820,000000,0
5,The Joint Chiropractic - San Tan Village,"2765 S. Market St., Ste 105","Gilbert, AZ 85295",-111.743563,33.300936,57366b8b498e4086cc2f8b70,0
6,The Joint Chiropractic - Gilbert Town Square,1084 S. Gilbert Road #103,"Gilbert, AZ 85296",-111.790251,33.330003,000000,0
7,The Joint Chiropractic - Arrowhead Ranch,"20280 N. 59th Avenue, Suite #107","Glendale, AZ 85308",-112.186889,33.669916,4bbfd52e74a9a593033dcff6,0
8,The Joint Chiropractic - Goodyear,"1468 N Litchfield Road, Suite M-6","Goodyear, AZ 85395",-112.358289,33.462838,58b8d5682f91cb60484ceac9,0
9,The Joint Chiropractic - Dana Park,"3426 E. Baseline Road, Suite #116","Mesa, AZ 85204",-111.756936,33.379282,4b4d17ccf964a520acca26e3,0


#### Extract the clinics that have at least 5 Likes

In [19]:
df_joint[df_joint['Like Count'] >=5]

,Location,Street Address,Zip Code,Longitude,Latitude,Venue_ID,Like Count
44,The Joint Chiropractic - Chino Hills,"13920 City Center Dr., Suite 4003","Chino Hills, CA 91709",-117.730268,34.000583,4cfab261c671721ec00fbf18,140
52,The Joint Chiropractic - El Cajon,100 Fletcher Pkwy,"El Cajon, CA 92020",-116.962592,32.807867,4aa99db1f964a520855420e3,38
99,The Joint Chiropractic - The Plant,"111 Curtner Ave, Suite #40","San Jose, CA 95125",-121.863817,37.301953,4bbbcb99afe1b713b7b62f4b,57
189,The Joint Chiropractic - The Collection at For...,"410 Peachtree Parkway, Suite 4122","Cumming, GA 30041",-84.174286,34.153837,4b7ad32af964a520ff3e2fe3,82
204,The Joint Chiropractic - Peachtree Corners,"5185 Peachtree Parkway, Suite 104-B","Peachtree Corners, GA 30092",-84.216483,33.979674,4a5766dbf964a52013b61fe3,163
231,The Joint Chiropractic - Hamilton Town Center,14191 Town Center Blvd. Suite #100,"Noblesville, IN 46060",-85.931593,39.994322,4bd9dbd93f01c9b65c6553a5,158
348,The Joint Chiropractic - Mt. Juliet,401 S Mt. Juliet Road Suite 245,"Mt. Juliet, TN 37122",-86.514309,36.166591,4e32f7c9b3ad90171c62d542,6
356,The Joint Chiropractic - Arboretum-Austin,"10710 Research Blvd, Suite #112","Austin, TX 78759",-97.746615,30.397988,58b206d65804ea3b2a3e0893,5
370,The Joint Chiropractic - Corpus Christi,"5425 South Padre Island Drive, Suite #146","Corpus Christi, TX 78411",-97.375764,27.710137,4b899316f964a520f84232e3,60
409,The Joint Chiropractic - Magnolia,"6519 FM 1488, Suite #513","Magnolia, TX 77354",-95.565589,30.221784,54e6789d498e7465ffeefa91,7


#### Since only 12 of 474 clinics has accumulated greater than 5 likes . We will check the Google Places API as an alternative. Foursquare does not have enough ratings data.